In [42]:
from pprint import pprint
import urllib
from bs4 import BeautifulSoup
from webdriver_manager.firefox import GeckoDriverManager
import time
import sys
import datetime

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [36]:
original_file_descriptor = sys.stdout

In [37]:
sys.stdout = open("./scrape_log.txt", "a")

In [49]:
print("------------- STARTING ON " + str(datetime.datetime.now()) + " -------------")

In [46]:
# Given a URL string, opens the URL in a headless Firefox instance and returns a reference to the webdriver object

def obtain_driver(url):
    buttons = []
    driver_options = Options()
    driver_options.headless = True
    
    print("***" + str(datetime.datetime.now()) + "***")
    driver = webdriver.Firefox(options=driver_options, executable_path=GeckoDriverManager().install())
    # driver.install_addon("~/.mozilla/firefox/f4t6w0s5.default-release/extensions/uBlock0@raymondhill.net.xpi", temporary=True)

    driver.get(url)
    
    time.sleep(1)
    
    return driver

In [32]:
# Given a DOI, uses obtain_driver to open up the search site and search the DOI to open up the paper in the page. Returns the webdriver associated with it.

def search_paper(doi):
    driver = obtain_driver("https://www.researchgate.net/search")
    
    search_field = driver.find_elements_by_class_name("search-form__input")[0]
    search_field.send_keys(doi)
    
#     time.sleep(1)
    
    enter_field = driver.find_elements_by_class_name("search-form__left-button")[0]
    driver.execute_script("arguments[0].click()", enter_field)
    
    time.sleep(1)
    
    if "www.researchgate.net/search/" in driver.current_url:
        print("Paper not found (search)")
        driver.quit()
        return None
    else: 
        return driver

In [29]:
# Given the webdriver, shows more authors. Returns the modified webdriver.

def show_authors(driver): 
    
    time.sleep(1)
    
    if not driver:
        print("Exiting: No driver")
        return 0
    
    first_start = time.time()
    
    buttons = driver.find_elements_by_class_name("show-more-authors")
    
    print("show_more_authors button")
    print(buttons)
    
    # Need to address the condition of the button not existing (i.e. <= 4 authors for paper)
    
    if len(buttons) == 1:
        time.sleep(3)
        print("click")
        driver.execute_script("arguments[0].click();", buttons[0])
#         WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "show-less-authors")))
        time.sleep(3)    
    
    first_end = time.time()

    print("DONE - " + str(first_end-first_start))
    
    soup_it(driver)
    
    return 1
    
# driver.quit()


In [47]:
# Given the webdriver, parses its source for author URLs.

def soup_it(driver):
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    thing = soup.find_all('div', {"class": "nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit nova-e-text--clamp nova-v-person-list-item__title"})
    for stuff in thing:
        print(stuff.find('a').get("href"))
    
    driver.quit()

In [50]:
# Testing on an example

# source = search_paper("10.1021/acs.iecr.5b03509")   # 6 authors
source = search_paper("10.1007/s10450-014-9639-3")  # 9 authors
# source = search_paper("10.1002/adfm.200500561")     # 4 authors
# source = search_paper("10.1002/adfm.200500563")     # doesn't exist

# show_authors(source)

# if exit_val != 0:
searched = show_authors(source)
# soup_it(searched)

In [8]:
# Obtains author URLs from a basic request. Can't interact with the webpage to show more authors if there are > 4 authors.

def obtain_author_researchgate(website_url):
    request = urllib.request.Request(
        url = website_url,
        data = None,
        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:67.0) Gecko/20100101 Firefox/67.0'
        }
    )

    testurl = urllib.request.urlopen(request)
    soup = BeautifulSoup(testurl, 'html.parser')

    thing = soup.find_all('div', {"class": "nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit nova-e-text--clamp nova-v-person-list-item__title"})
    for stuff in thing:
        print(stuff.find('a').get("href"))